In [1]:
import mne
import numpy as np
import matplotlib.pyplot as plt

# Import the FOOOF object
from fooof import FOOOF
from fooof import FOOOFGroup
# Import a utility to download and load example data
from fooof.utils.download import load_fooof_data


In [2]:
EEG = mne.read_epochs_eeglab('/home/b1044271/Columbia/Test_dataset.set')


/home/b1044271/anaconda3/envs/mne/lib/python3.10/site-packages/pymatreader/utils.py:122: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


Extracting parameters from /home/b1044271/Columbia/Test_dataset.set...


/tmp/ipykernel_3612263/4273115982.py:1: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  EEG = mne.read_epochs_eeglab('/home/b1044271/Columbia/Test_dataset.set')


Not setting metadata
50 matching events found
No baseline correction applied
0 projection items activated
Ready.


In [3]:
EEG2 = EEG.pick(['E1','E2','E3','E257'], exclude=[]) # select Cz for further analysis


In [18]:
np.size(EEG2,1)

4

In [4]:
#Now we select epochs and average over all epochs
N1s = EEG2['stage_1']
N2s = EEG2['stage_2']
N3s = EEG2['stage_3']
REs = EEG2['stage_5']
AWs = EEG2['stage_0']
Time_segments = [2, 5, 10, 15] #in seconds
f_max = [30, 45 , 55, 70, 100]
fs = 250
SETTINGS_B = { 'method' : 'welch', 'average' : 'mean', 'fmin' :1 }

    # Frequency transformation welch's method from 1 to 70Hz
N1={}
N2={}
N3={}
AW={}
RM={}

i = 0
for x in Time_segments:
    for y in f_max:
        i = i+1
        N1[i] = N1s.compute_psd(**SETTINGS_B, fmax=y, n_fft=fs*x) #N1
        N2[i] = N2s.compute_psd(**SETTINGS_B, fmax=y, n_fft=fs*x) #N2
        N3[i] = N3s.compute_psd(**SETTINGS_B, fmax=y, n_fft=fs*x)
        AW[i] = AWs.compute_psd(**SETTINGS_B, fmax=y, n_fft=fs*x)
        RM[i] = REs.compute_psd(**SETTINGS_B, fmax=y, n_fft=fs*x)




Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 2.000 (s)
Effective window size : 5.000 (s)
Effective window size : 5.000 (s)
Effective window size : 5.000 (s)
Effective window size : 5.000 (s)
Effective wind

In [5]:
# model fit on different freq characteristics
SETTINGS_F1={'max_n_peaks':8, 'aperiodic_mode':'fixed'}
SETTINGS_F2={'max_n_peaks':8, 'aperiodic_mode':'knee'}
path_results = '/home/b1044271/Columbia/Files/'
timesteps = ["2","5","10","15"]*5

fooof_a = {}
fooof_b = {}

i = 0
variables = [N1, N2, N3, AW, RM]
var_n = np.repeat(np.array(["N1", "N2", "N3","R","W"]), 4, axis=0)
for A in variables:
    for keys, values in A.items(): 
        fm1 = FOOOFGroup(**SETTINGS_F1)
        fm1.fit(A[keys]._freqs, np.mean(np.squeeze(A[keys]._data),0), [A[keys]._freqs[0] , A[keys]._freqs[-1]])
        fm1.save(var_n[i]+'_'+str(round(A[keys]._freqs[-1]))+timesteps[i]+'_NoKnee',  file_path = path_results, save_results=True)

        fm2 = FOOOFGroup(**SETTINGS_F2)
        fm2.fit(A[keys]._freqs, np.mean(np.squeeze(A[keys]._data),0), [A[keys]._freqs[0] , A[keys]._freqs[-1]])
        fm2.save(var_n[i]+'_'+str(round(A[keys]._freqs[-1]))+timesteps[i]+'_Knee', file_path = path_results, save_results=True)
        fooof_b[i] = fm2.get_results()

        i=i+1

Running FOOOFGroup across 4 power spectra.

FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.50 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.

Running FOOOFGroup across 4 power spectra.

FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.50 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.

Running FOOOFGroup across 4 power spectra.

FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.50 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower

/home/b1044271/anaconda3/envs/mne/lib/python3.10/site-packages/fooof/core/funcs.py:67: RuntimeWarning: invalid value encountered in log10
  ys = ys + offset - np.log10(knee + xs**exp)
/home/b1044271/anaconda3/envs/mne/lib/python3.10/site-packages/fooof/core/funcs.py:67: RuntimeWarning: invalid value encountered in log10
  ys = ys + offset - np.log10(knee + xs**exp)


Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.
Running FOOOFGroup across 4 power spectra.


KeyboardInterrupt: 

In [8]:
type(variables)
type(N1)

dict

In [18]:
var_n[i]+'_'+str(round(A[keys]._freqs[-1]))+'_'+timesteps[i]+'_NoKnee'
cd

NameError: name 'cd' is not defined

In [13]:
FOOOF.save?

In [18]:
timesteps = ["2","5","10","15"]*5
timesteps[1]
var_n[1]

'N2'

In [13]:
variables[0]

{1: <Power Spectrum (from Epochs, welch method) | 14 epochs × 4 channels × 59 freqs, 1.0-30.0 Hz>,
 2: <Power Spectrum (from Epochs, welch method) | 14 epochs × 4 channels × 89 freqs, 1.0-45.0 Hz>,
 3: <Power Spectrum (from Epochs, welch method) | 14 epochs × 4 channels × 109 freqs, 1.0-55.0 Hz>,
 4: <Power Spectrum (from Epochs, welch method) | 14 epochs × 4 channels × 139 freqs, 1.0-70.0 Hz>,
 5: <Power Spectrum (from Epochs, welch method) | 14 epochs × 4 channels × 199 freqs, 1.0-100.0 Hz>,
 6: <Power Spectrum (from Epochs, welch method) | 14 epochs × 4 channels × 146 freqs, 1.0-30.0 Hz>,
 7: <Power Spectrum (from Epochs, welch method) | 14 epochs × 4 channels × 221 freqs, 1.0-45.0 Hz>,
 8: <Power Spectrum (from Epochs, welch method) | 14 epochs × 4 channels × 271 freqs, 1.0-55.0 Hz>,
 9: <Power Spectrum (from Epochs, welch method) | 14 epochs × 4 channels × 346 freqs, 1.0-70.0 Hz>,
 10: <Power Spectrum (from Epochs, welch method) | 14 epochs × 4 channels × 496 freqs, 1.0-100.0 Hz>,

In [6]:
cd

/home/b1044271


In [ ]:
label = 'N1'
subj = 12

fg.save(label + '_' + str(subj) + '_res')

In [61]:
from fooof.objs.utils import combine_fooofs

In [62]:
fm1.save?

In [55]:
fm1.save('test_save', file_path=PATH_TO_RESULTS_FOLDER, save_results=True)

In [57]:
fg_new = FOOOFGroup()

In [58]:
fg_new.load('test_save')

In [60]:
fg_new.get_params('aperiodic', 'exponent')

array([2.16666491, 2.16316968, 2.08798823, 2.22394343])

In [56]:
!ls -l

total 3738756
-rw-r--r--  1 b1044271 b1094405  690850711 May 10  2022  Anaconda3-2022.05-Linux-x86_64.sh
-rw-r--r--  1 b1044271 b1094405  690850711 May 10  2022  Anaconda3-2022.05-Linux-x86_64.sh.1
-rw-r--r--  1 b1044271 b1094405     868657 Dec  8 00:25  Analysis_FOOFGroup.ipynb
-rw-r--r--  1 b1044271 b1094405    3275603 Dec  7 19:59  Analysis_first.ipynb
drwxr-xr-x 10 b1044271 b1094405       4096 May 30  2022  BABY
-rw-r--r--  1 b1044271 b1094405        613 Aug 20 03:15  BBA_NEMs.txt
-rw-r--r--  1 b1044271 b1094405      17287 Nov 27 08:28  Bars.svg
-rw-r--r--  1 b1044271 b1094405     417714 Nov 27 08:16  Bars.tiff
-rw-r--r--  1 b1044271 b1094405       1159 Nov 26 03:41  Coh_Beta_REM_C_4s.txt
-rw-r--r--  1 b1044271 b1094405       1162 Nov 21 08:30  Coh_Beta_REM_all.txt
-rw-r--r--  1 b1044271 b1094405       1155 Nov 26 02:57  Coh_Beta_REM_all4s.txt
-rw-r--r--  1 b1044271 b1094405       1158 Nov 26 02:57  Coh_Theta_REM_all4s.txt
drwxr-xr-x  4 b1044271 b1094405       4096 Nov 30 06:05  Co

In [53]:
fm1.get_params('peak')

array([[ 6.48313498,  0.24535206,  4.48681495,  0.        ],
       [ 9.47677294,  0.27758353,  0.8451313 ,  0.        ],
       [10.35941263,  0.31400104,  0.5       ,  0.        ],
       [12.20022596,  0.33188276,  2.32888926,  0.        ],
       [13.70756974,  0.87703602,  0.82133657,  0.        ],
       [14.58333333,  0.70438072,  1.85460023,  0.        ],
       [21.59278582,  0.34049733,  0.51869222,  0.        ],
       [25.11666667,  0.12204601, 11.99999932,  0.        ],
       [ 7.66476574,  0.24854591,  4.12458066,  1.        ],
       [12.43086894,  0.46908002,  0.5       ,  1.        ],
       [12.48333168,  0.46958083,  4.04309826,  1.        ],
       [13.91287121,  0.77648792,  0.8463044 ,  1.        ],
       [14.86455633,  0.60198745,  0.76852574,  1.        ],
       [17.41666667,  0.18364001, 12.        ,  1.        ],
       [25.74893757,  0.29749931,  0.5       ,  1.        ],
       [27.27576882,  0.25202152,  0.99155694,  1.        ],
       [ 6.85001673,  0.

In [ ]:
all_exps = [fr.aperiodic_params[2] for fr in fm1.group_results]

In [34]:
fr_lst[0].aperiodic_params

array([-10.870244  ,  -0.79998956,   1.19709313])

In [35]:
all_exps = [fr.aperiodic_params[2] for fr in fr_lst]

In [37]:
all_exps2 = []
for fr in fr_lst:
    all_exps2.append(fr.aperiodic_params[2])

In [38]:
all_exps2

[1.1970931312844504, 0.8986240392523056, 1.082676305210807, 1.127071536618664]

In [36]:
all_exps

[1.1970931312844504, 0.8986240392523056, 1.082676305210807, 1.127071536618664]

In [32]:
fm1.group_results[0]

FOOOFResults(aperiodic_params=array([-10.07051718,   2.16666491]), peak_params=array([[ 6.48313498,  0.24535206,  4.48681495],
       [ 9.47677294,  0.27758353,  0.8451313 ],
       [10.35941263,  0.31400104,  0.5       ],
       [12.20022596,  0.33188276,  2.32888926],
       [13.70756974,  0.87703602,  0.82133657],
       [14.58333333,  0.70438072,  1.85460023],
       [21.59278582,  0.34049733,  0.51869222],
       [25.11666667,  0.12204601, 11.99999932]]), r_squared=0.972603999307042, error=0.11242944898461407, gaussian_params=array([[ 6.48313498,  0.2443832 ,  2.24340748],
       [ 9.47677294,  0.1540955 ,  0.42256565],
       [10.35941263,  0.15365992,  0.25      ],
       [12.20022596,  0.28744698,  1.16444463],
       [13.70756974,  0.33326804,  0.41066828],
       [14.58333333,  0.61205453,  0.92730011],
       [21.59278582,  0.23780438,  0.25934611],
       [25.11666667,  0.12204648,  5.99999966]]))

In [21]:
fooof_b

{0: [FOOOFResults(aperiodic_params=array([-10.870244  ,  -0.79998956,   1.19709313]), peak_params=array([[ 3.68443723,  0.12044423,  0.5       ],
         [11.59550932,  0.18144924,  1.80481692],
         [13.87996812,  0.24624565,  1.16982491],
         [24.10692885,  0.20240341,  4.20003521],
         [27.06207731,  0.17031704,  1.85633873]]), r_squared=0.9928011575961432, error=0.031136914980592286, gaussian_params=array([[ 3.68443723,  0.15811515,  0.25      ],
         [11.59550932,  0.18240549,  0.90240846],
         [13.87996812,  0.24613188,  0.58491246],
         [24.10692885,  0.20226602,  2.1000176 ],
         [27.06207731,  0.09221615,  0.92816937]])),
  FOOOFResults(aperiodic_params=array([-11.11101054,  -0.91310754,   0.89862404]), peak_params=array([[ 2.38263859,  0.12894966,  2.28081149],
         [11.65181111,  0.13181565,  1.28070344],
         [13.821179  ,  0.14280359,  0.76105349],
         [19.64794272,  0.12818397,  1.17861409],
         [21.98967564,  0.28768732

In [30]:
fr_lst

[FOOOFResults(aperiodic_params=array([-10.870244  ,  -0.79998956,   1.19709313]), peak_params=array([[ 3.68443723,  0.12044423,  0.5       ],
        [11.59550932,  0.18144924,  1.80481692],
        [13.87996812,  0.24624565,  1.16982491],
        [24.10692885,  0.20240341,  4.20003521],
        [27.06207731,  0.17031704,  1.85633873]]), r_squared=0.9928011575961432, error=0.031136914980592286, gaussian_params=array([[ 3.68443723,  0.15811515,  0.25      ],
        [11.59550932,  0.18240549,  0.90240846],
        [13.87996812,  0.24613188,  0.58491246],
        [24.10692885,  0.20226602,  2.1000176 ],
        [27.06207731,  0.09221615,  0.92816937]])),
 FOOOFResults(aperiodic_params=array([-11.11101054,  -0.91310754,   0.89862404]), peak_params=array([[ 2.38263859,  0.12894966,  2.28081149],
        [11.65181111,  0.13181565,  1.28070344],
        [13.821179  ,  0.14280359,  0.76105349],
        [19.64794272,  0.12818397,  1.17861409],
        [21.98967564,  0.28768732,  1.77199986],
 

In [28]:
fr

FOOOFResults(aperiodic_params=array([-10.870244  ,  -0.79998956,   1.19709313]), peak_params=array([[ 3.68443723,  0.12044423,  0.5       ],
       [11.59550932,  0.18144924,  1.80481692],
       [13.87996812,  0.24624565,  1.16982491],
       [24.10692885,  0.20240341,  4.20003521],
       [27.06207731,  0.17031704,  1.85633873]]), r_squared=0.9928011575961432, error=0.031136914980592286, gaussian_params=array([[ 3.68443723,  0.15811515,  0.25      ],
       [11.59550932,  0.18240549,  0.90240846],
       [13.87996812,  0.24613188,  0.58491246],
       [24.10692885,  0.20226602,  2.1000176 ],
       [27.06207731,  0.09221615,  0.92816937]]))

In [29]:
fr_lst = fooof_b[0]
fr = fooof_b[0][0]

In [10]:
fm1 = FOOOFGroup(**SETTINGS_F1)
fm1.fit(A[1]._freqs, np.mean(np.squeeze(A[1]._data),0), freq_range)
fooof_a = fm1.get_results()


fooof_a

Running FOOOFGroup across 4 power spectra.

FOOOF WARNING: Lower-bound peak width limit is < or ~= the frequency resolution: 0.50 <= 0.50
	Lower bounds below frequency-resolution have no effect (effective lower bound is the frequency resolution).
	Too low a limit may lead to overfitting noise as small bandwidth peaks.
	We recommend a lower bound of approximately 2x the frequency resolution.



[FOOOFResults(aperiodic_params=array([-10.0598074 ,   2.00723522]), peak_params=array([[ 5.31944968,  0.12547147,  2.57581478],
        [ 7.38689363,  0.16956962,  1.22952939],
        [ 9.86800181,  0.20281065,  0.90477839],
        [11.35910531,  0.19549845,  1.19273167],
        [13.84248703,  0.77842712,  1.63104811],
        [16.37792375,  0.15997938,  1.07953351]]), r_squared=0.9961233457215165, error=0.03253134652357749, gaussian_params=array([[ 5.31944968,  0.12541921,  1.28790739],
        [ 7.38689363,  0.14203768,  0.6147647 ],
        [ 9.86800181,  0.1968414 ,  0.45238919],
        [11.35910531,  0.18754979,  0.59636584],
        [13.84248703,  0.79306222,  0.81552406],
        [16.37792375,  0.16010056,  0.53976675]])),
 FOOOFResults(aperiodic_params=array([-9.82075293,  2.11615942]), peak_params=array([[ 7.61349724,  0.14242684,  2.52255757],
        [ 9.92101163,  0.16706327,  1.26282225],
        [12.21649387,  0.24814808,  2.47550349],
        [14.01536388,  0.6581636

In [32]:
fooof_a

{0: [FOOOFResults(aperiodic_params=array([-10.56426689,   1.44272318]), peak_params=array([[11.65165908,  0.18068333,  1.29663429],
         [13.83018802,  0.23707663,  1.21712366],
         [24.08131858,  0.22386987,  4.49485326],
         [27.19173244,  0.19951967,  2.3342263 ]]), r_squared=0.9768333297650668, error=0.04890333221597683, gaussian_params=array([[11.65165908,  0.18552945,  0.64831715],
         [13.83018802,  0.24620542,  0.60856183],
         [24.08131858,  0.22150423,  2.24742663],
         [27.19173244,  0.10562289,  1.16711315]])),
  FOOOFResults(aperiodic_params=array([-10.74945351,   1.18869116]), peak_params=array([[ 1.        ,  0.7599179 ,  0.76955749],
         [11.74436741,  0.12934077,  0.93579202],
         [13.82135366,  0.15088725,  0.83037347],
         [21.77661333,  0.31072901,  3.42238053],
         [26.54802836,  0.33231431,  5.24781924]]), r_squared=0.9863067288980146, error=0.036452835404511726, gaussian_params=array([[ 1.        ,  0.7599179 ,  0.

In [23]:
FOOOF.save?

In [34]:
fx=FOOOF()
fx.load('W_100.015_NoKnee')

JSONDecodeError: Extra data: line 2 column 1 (char 972)

In [32]:
cd

/home/b1044271


In [38]:
fx=FOOOF()

In [42]:
fx.load('/home/b1044271/Columbia/Files/W_10015_NoKnee.json')

JSONDecodeError: Extra data: line 2 column 1 (char 972)

In [40]:
cd

/home/b1044271
